# Generates the scaled down images

In [12]:
from ocdc_dataloader import *

magnification=0.625
scale = get_scale_by_magnification(magnification)
tile_size=20
tile_size_original = int(scale * tile_size)


cls = "tumor"
dataset_types = ["training", "testing"]
for dataset_type in sorted(dataset_types):
    
    logger.info("{} images".format(dataset_type))
    
    ocdc_dir = "../../datasets/OCDC"
    wsi_images_dir = "{}/{}/{}/wsi".format(ocdc_dir, dataset_type, cls)
    patch_images_dir = "{}/{}/{}/patch/640x640".format(ocdc_dir, dataset_type, cls)
    
    for r, d, f in sorted(os.walk(wsi_images_dir)):
        for wsi_file in sorted(f):
            
            wsi_image_file = "{}/{}".format(r, wsi_file)
            wsi_image_number = wsi_file.replace(".svs", "")

            file_is_svs = wsi_image_file.lower().endswith('.svs')
            if file_is_svs:

                logger.info("Processing wsi '{}'".format(wsi_file))
                
                # check directory to save image-patches
                dir_to_save = "{}/{}".format(patch_images_dir, wsi_image_number)
                if not os.path.exists(dir_to_save):
                    os.makedirs("{}/01-roi/01-original".format(dir_to_save))
                    os.makedirs("{}/01-roi/02-mask".format(dir_to_save))
                    os.makedirs("{}/01-roi/03-roi".format(dir_to_save))             
                    os.makedirs("{}/02-non_roi/01-original".format(dir_to_save))
                    os.makedirs("{}/02-non_roi/02-mask".format(dir_to_save))
                    
                for f in sorted(os.listdir("{}/01-roi/01-original".format(dir_to_save))):
                    
                    image_patch_file = "{}/01-roi/01-original/{}".format(dir_to_save, f)
                    pil_tile = load_pil_image(image_patch_file, gray=False)
                    pil_tile_mask = load_pil_image(image_patch_file.replace("01-original", "02-mask"), gray=True)
                    pil_tile_roi = blend_image(pil_tile, pil_tile_mask, foreground='blue', alpha=0.6, inverse=True)                  
                    pil_tile_roi.save("{}/01-roi/03-roi/{}".format(dir_to_save, f))
                    
                
                # wsi image
                wsi_image = open_wsi(wsi_image_file)
                pil_scaled_down_image = scale_down_wsi(wsi_image, magnification, False)
                np_scaled_down_image = pil_to_np(pil_scaled_down_image)
                
                # extract tissue region 
                np_tissue_mask, np_masked_image = extract_normal_region_from_wsi(wsi_image_file, np_scaled_down_image, None)
                np_normal_mask = np.zeros((np_scaled_down_image.shape[0], np_scaled_down_image.shape[1]), dtype=bool)
                np_normal_mask[np_tissue_mask > 0] = True
                np_normal_mask[np_tissue_mask == 0] = False
                pil_normal_mask = np_to_pil(np_normal_mask)
                
                pil_normal_mask = np_to_pil(np_normal_mask)
                pil_normal_masked = np_to_pil(np_masked_image)
                
                blend_tissue = blend_image(pil_scaled_down_image, pil_normal_mask, foreground='green', inverse=True)
                blend_background = blend_image(blend_tissue, pil_normal_mask, foreground='blue', inverse=False)

                #heat grid tissue
                pil_img_normal_result, heat_grid_normal, number_of_tiles_normal = draw_heat_grid(np_masked_image, tile_size)
                
                
                pil_scaled_down_image.save('{}/{}_1.png'.format(dir_to_save, wsi_image_number))
                blend_background.save('{}/{}_2.png'.format(dir_to_save, wsi_image_number))
                pil_normal_masked.save('{}/{}_3.png'.format(dir_to_save, wsi_image_number))
                pil_img_normal_result.save('{}/{}_5.png'.format(dir_to_save, wsi_image_number))
                

2021-02-02 09:08:42,813 :: INFO <module> :: testing images
2021-02-02 09:08:42,817 :: INFO <module> :: Processing wsi 'TMA HE.svs'
2021-02-02 09:08:43,646 :: INFO extract_normal_region_from_wsi :: 	 Extracting normal regions from wsi image: 'TMA HE.svs'
2021-02-02 09:08:48,900 :: INFO <module> :: training images
2021-02-02 09:08:48,902 :: INFO <module> :: Processing wsi '1009009.svs'
2021-02-02 09:09:04,002 :: INFO extract_normal_region_from_wsi :: 	 Extracting normal regions from wsi image: '1009009.svs'
2021-02-02 09:09:04,557 :: INFO <module> :: Processing wsi '1009010x1000902.svs'
2021-02-02 09:09:17,272 :: INFO extract_normal_region_from_wsi :: 	 Extracting normal regions from wsi image: '1009010x1000902.svs'
2021-02-02 09:09:19,345 :: INFO <module> :: Processing wsi '1009011.svs'
2021-02-02 09:09:37,403 :: INFO extract_normal_region_from_wsi :: 	 Extracting normal regions from wsi image: '1009011.svs'
2021-02-02 09:09:38,550 :: INFO <module> :: Processing wsi '1009014.svs'
2021-0